# Tfrecord

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
from tensorflow import keras
print('tf    version: {}'.format(tf.__version__) )
print('keras version: {}'.format(keras.__version__) )
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

tf    version: 2.0.0
keras version: 2.2.4-tf
matplotlib 3.0.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
favorite_books = [name.encode('utf-8')  for name in  ["machine learning", "cc150"]]

In [3]:
favorite_books_bytelist = tf.train.BytesList(value = favorite_books)

In [4]:
print(favorite_books_bytelist)

value: "machine learning"
value: "cc150"



In [5]:
hours_floatlist = tf.train.FloatList(value = [15.5, 9, 7.0,  8.0])
age_int64list = tf.train.Int64List(value = [42])
print(hours_floatlist)
print(age_int64list)

features = tf.train.Features(
    feature = {
        "favorite_books": tf.train.Feature(bytes_list = favorite_books_bytelist),
        "hours": tf.train.Feature(float_list = hours_floatlist), 
        "age": tf.train.Feature(int64_list = age_int64list)
        
        
    }
)

print(features)

value: 15.5
value: 9.0
value: 7.0
value: 8.0

value: 42

feature {
  key: "age"
  value {
    int64_list {
      value: 42
    }
  }
}
feature {
  key: "favorite_books"
  value {
    bytes_list {
      value: "machine learning"
      value: "cc150"
    }
  }
}
feature {
  key: "hours"
  value {
    float_list {
      value: 15.5
      value: 9.0
      value: 7.0
      value: 8.0
    }
  }
}



In [6]:
example = tf.train.Example(features= features)
print(example)
serialized_example = example.SerializeToString()
print(serialized_example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 42
      }
    }
  }
  feature {
    key: "favorite_books"
    value {
      bytes_list {
        value: "machine learning"
        value: "cc150"
      }
    }
  }
  feature {
    key: "hours"
    value {
      float_list {
        value: 15.5
        value: 9.0
        value: 7.0
        value: 8.0
      }
    }
  }
}

b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x10A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'


In [7]:
output_dir = './output/tfrecord'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
filename = "test.tfrecords"
filename_fullpath = os.path.join(output_dir, filename)

with tf.io.TFRecordWriter(filename_fullpath) as writer:
    for i in range(3):
        writer.write(serialized_example)

In [8]:
dataset = tf.data.TFRecordDataset([filename_fullpath])

for serialized_example_tensor in dataset:
    print(serialized_example)

b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x10A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'
b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x10A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'
b'\n\\\n\x1d\n\x05hours\x12\x14\x12\x12\n\x10\x00\x00xA\x00\x00\x10A\x00\x00\xe0@\x00\x00\x00A\n-\n\x0efavorite_books\x12\x1b\n\x19\n\x10machine learning\n\x05cc150\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01*'


In [9]:
expected_features = {
    "favorite_books": tf.io.VarLenFeature(dtype = tf.string), 
    "hours": tf.io.VarLenFeature(dtype = tf.float32), 
    "age": tf.io.FixedLenFeature([], dtype= tf.int64),
}
print(expected_features)

{'favorite_books': VarLenFeature(dtype=tf.string), 'hours': VarLenFeature(dtype=tf.float32), 'age': FixedLenFeature(shape=[], dtype=tf.int64, default_value=None)}


In [10]:
dataset = tf.data.TFRecordDataset([filename_fullpath])

for serialized_example_tensor in dataset:
    example = tf.io.parse_single_example(
        serialized_example_tensor, 
        expected_features
    )
    books = tf.sparse.to_dense(example["favorite_books"], 
                              default_value="b")
    for book in books:
        print(book.numpy().decode('UTF-8'))
    


machine learning
cc150
machine learning
cc150
machine learning
cc150


In [11]:
filename_fullpath_zip = filename_fullpath + '.zip'

options = tf.io.TFRecordOptions(compression_type="GZIP")

with tf.io.TFRecordWriter(filename_fullpath_zip, options) as writer:
    for i in range(3):
        writer.write(serialized_example)

In [12]:
dataset = tf.data.TFRecordDataset([filename_fullpath_zip], 
                                 compression_type="GZIP")

for serialized_example_tensor in dataset:
    example = tf.io.parse_single_example(
        serialized_example_tensor, 
        expected_features
    )
    books = tf.sparse.to_dense(example["favorite_books"], 
                              default_value="b")
    for book in books:
        print(book.numpy().decode('UTF-8'))
    


machine learning
cc150
machine learning
cc150
machine learning
cc150


In [13]:
source_dir = './dataset/'
def get_filenames_by_prefix(source_dir, prefix_name):
    all_files = os.listdir(source_dir)
    results = []
    for filename in all_files:
        if filename.startswith(prefix_name):
            results.append(os.path.join(source_dir, filename))
            
    return results
train_filenames = get_filenames_by_prefix(source_dir, "train")
valid_filenames = get_filenames_by_prefix(source_dir, "valid")
test_filenames = get_filenames_by_prefix(source_dir, "test")
print(train_filenames)

['./dataset/train_11.csv', './dataset/train_09.csv', './dataset/train_13.csv', './dataset/train_06.csv', './dataset/train_01.csv', './dataset/train_12.csv', './dataset/train_17.csv', './dataset/train_07.csv', './dataset/train_04.csv', './dataset/train_15.csv', './dataset/train_00.csv', './dataset/train_16.csv', './dataset/train_10.csv', './dataset/train_03.csv', './dataset/train_05.csv', './dataset/train_08.csv', './dataset/train_14.csv', './dataset/train_02.csv', './dataset/train_19.csv', './dataset/train_18.csv']


In [14]:
def parse_csv_line(line, n_fields = 9):
    defs = [tf.constant(np.nan)] * n_fields
    parsed_fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(parsed_fields[0:-1])
    y = tf.stack(parsed_fields[-1:])
    return x, y

def csv_reader_dataset(filenames, n_readers=5,
                       batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_csv_line,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,
                              batch_size = batch_size)

In [15]:
print(type(test_set))

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [16]:
def serialize_example(x, y):
    """Converts x, y to tf.train.Example and serialize"""
    input_feautres = tf.train.FloatList(value = x)
    label = tf.train.FloatList(value = y)
    features = tf.train.Features(
        feature = {
            "input_features": tf.train.Feature(
                float_list = input_feautres),
            "label": tf.train.Feature(float_list = label)
        }
    )
    example = tf.train.Example(features = features)
    return example.SerializeToString()

In [17]:
def csv_dataset_to_tfrecords(base_filename, dataset, 
                            n_shards, steps_per_shard, 
                            compression_type= None):
    
    options = tf.io.TFRecordOptions(compression_type=compression_type)
    
    for shard_id in range(n_shards):
        filename_fullpath = '{}_{:05d}-of-{:05d}'.format(
            base_filename, shard_id, n_shards
        )
        all_filenames = []
        with tf.io.TFRecordWriter(filename_fullpath, options) as writer:
            
            for x_batch, y_batch in dataset.take(steps_per_shard):
                for x_example, y_example in zip(x_batch, y_batch):
                    writer.write(serialize_example(x_example, y_example))
    
        all_filenames.append(filename_fullpath)
    return all_filenames

In [23]:
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "./output/generate_tfrecords"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard, None)
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, n_shards, valid_steps_per_shard, None)
test_tfrecord_fielnames = csv_dataset_to_tfrecords(
    test_basename, test_set, n_shards, test_steps_per_shard, None)

In [21]:
print(train_tfrecord_filenames)
print(valid_tfrecord_filenames)
print(test_tfrecord_fielnames)

['generate_tfrecords/train_00019-of-00020']
['generate_tfrecords/valid_00019-of-00020']
['generate_tfrecords/test_00019-of-00020']


In [24]:
n_shards = 20
train_steps_per_shard = 11610 // batch_size // n_shards
valid_steps_per_shard = 3880 // batch_size // n_shards
test_steps_per_shard = 5170 // batch_size // n_shards

output_dir = "./output/generate_tfrecords_zip"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

train_basename = os.path.join(output_dir, "train")
valid_basename = os.path.join(output_dir, "valid")
test_basename = os.path.join(output_dir, "test")

train_tfrecord_filenames = csv_dataset_to_tfrecords(
    train_basename, train_set, n_shards, train_steps_per_shard,
    compression_type = "GZIP")
valid_tfrecord_filenames = csv_dataset_to_tfrecords(
    valid_basename, valid_set, n_shards, valid_steps_per_shard,
    compression_type = "GZIP")
test_tfrecord_fielnames = csv_dataset_to_tfrecords(
    test_basename, test_set, n_shards, test_steps_per_shard,
    compression_type = "GZIP")

In [25]:
expected_features = {
    "input_features": tf.io.FixedLenFeature([8], dtype=tf.float32),
    "label": tf.io.FixedLenFeature([1], dtype=tf.float32)
}

def parse_example(serialized_example):
    example = tf.io.parse_single_example(serialized_example,
                                         expected_features)
    return example["input_features"], example["label"]

def tfrecords_reader_dataset(filenames, n_readers=5,
                             batch_size=32, n_parse_threads=5,
                             shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TFRecordDataset(
            filename, compression_type = "GZIP"),
        cycle_length = n_readers
    )
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(parse_example,
                          num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset

tfrecords_train = tfrecords_reader_dataset(train_tfrecord_filenames,
                                           batch_size = 3)
for x_batch, y_batch in tfrecords_train.take(10):
    print(x_batch)
    print(y_batch)

tf.Tensor(
[[-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]
 [-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]
 [-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[1.832]
 [1.832]
 [1.832]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]
 [-0.097193   -1.2497431   0.36232963  0.02690608  1.0338118   0.04588159
   1.3418335  -1.635387  ]
 [ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]], shape=(3, 8), dtype=float32)
tf.Tensor(
[[1.832]
 [1.832]
 [2.419]], shape=(3, 1), dtype=float32)
tf.Tensor(
[[ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.19737554 -0.02272263
  -0.69240725  0.72652334]
 [ 0.63034356  1.8741661  -0.06713215 -0.12543367 -0.1973755

In [26]:
batch_size = 32
tfrecords_train_set = tfrecords_reader_dataset(
    train_tfrecord_filenames, batch_size = batch_size)
tfrecords_valid_set = tfrecords_reader_dataset(
    valid_tfrecord_filenames, batch_size = batch_size)
tfrecords_test_set = tfrecords_reader_dataset(
    test_tfrecord_fielnames, batch_size = batch_size)

In [27]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',
                       input_shape=[8]),
    keras.layers.Dense(1),
])
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(
    patience=5, min_delta=1e-2)]

history = model.fit(tfrecords_train_set,
                    validation_data = tfrecords_valid_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks)

Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 2s 5ms/step - loss: nan - val_loss: nan
Epoch 2/100
348/348 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 3/100
348/348 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 4/100
348/348 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
Epoch 5/100
348/348 [==============================] - 1s 3ms/step - loss: nan - val_loss: nan
